In [9]:

TRAIN_DATA_PATH = "data/train.txt"

In [10]:
# Sanity check, to ensure data is correct
def line_can_be_output(line):
    """
    Checks if line (list) is an output
    """
    for char in line:
        if char != '0' and char != '1': return False
        
    return True

train_file = open(TRAIN_DATA_PATH, "r")
file_lines = train_file.readlines()

for i in range(len(file_lines)):
    line = file_lines[i].replace("\n", "").split(" ")[:-1]
    if i % 32 > 16 and not line_can_be_output(line): # If it is a piece of output data
        print("Discrepancy in line " + str(i) + "!")
        
print(len(file_lines)/32)
train_file.close()

790.0


In [13]:
import numpy as np
from random import randint

"""
Returns data as an entire numpy array
"""
def matricize_data(file_path, num_data):
    data_file = open(file_path, "r")
    input_matrix = []
    positions_matrix = []
    label_matrix = []
    # Get data, 16 lines per
    for i in range(num_data):
        input_board = [[[int(number)] for number in data_file.readline().replace("\n", "").split(" ")[:-1]] 
                            for i in range(16)]
        possible_moves = [[[int(number)] for number in data_file.readline().replace("\n", "").split(" ")[:-1]] 
                        for i in range(16)]
        for y in range(16):
            for x in range(16):
                index_insert = randint(0, len(input_matrix)-1) if input_matrix != [] else 0
                input_matrix.insert(index_insert, input_board)
                positions_matrix.insert(index_insert, [x, y])
                label_matrix.insert(index_insert, possible_moves[y][x])
    
    data_file.close()
    return (np.array(input_matrix), np.array(positions_matrix), np.array(label_matrix), len(input_matrix))
    
"""
Generator that yields a piece of data
"""
def gen_matricize_data(file_path, num_data):
    data_file = open(file_path, "r")
    for i in range(num_data):
        input_matrix = [[[int(number)] for number in data_file.readline().replace("\n").split(" ")[:-1]] 
                            for i in range(16)]
        label_matrix = [[[int(number)] for number in data_file.readline().replace("\n").split(" ")[:-1]] 
                        for i in range(16)]
        yield (input_matrix, label_matrix)
    
    data_file.close()
    return

train_board, train_positions, label_matrix, train_length = matricize_data(TRAIN_DATA_PATH, 600)
print(train_board.shape)

(153600, 16, 16, 1)


In [28]:
from keras import layers
from keras import models

board_input = layers.Input(shape=(16, 16, 1))


Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_89 (Conv2D)           (None, 14, 14, 4)         40        
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 12, 12, 8)         296       
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 10, 10, 16)        1168      
_________________________________________________________________
max_pooling2d_53 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 512)               205312    
_________________________________________________________________
dense_17 (Dense)             (None, 1)               